<a href="https://colab.research.google.com/github/SnrPep/Mos/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

train_test_split AutoTokenizer AutoModelForSequenceClassification TrainingArguments Trainer DataCollatorWithPadding evaluate numpy os

In [ ]:
!python -m ipykernel install --user --name=colab-env


Installed kernelspec colab-env in C:\Users\Konstantin\AppData\Roaming\jupyter\kernels\colab-env


In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

D:\dfsd\colab-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
df = pd.read_csv(url)

df = df[df['label'].isin([0, 1])].reset_index(drop=True)
df = df.head(1500)


# Деление на train/val/test
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [3]:
# Токенизация
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(example):
    return tokenizer(example["tweet"], truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 11828.05 examples/s]


In [4]:
# Модель и аргументы обучения
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments(
    output_dir="results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [5]:
# Метрики
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
# Обучение
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

C:\Temp\ipykernel_9908\3629407656.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
D:\dfsd\colab-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=225, training_loss=0.18434410942925347, metrics={'train_runtime': 391.8909, 'train_samples_per_second': 9.186, 'train_steps_per_second': 0.574, 'total_flos': 36420255417984.0, 'train_loss': 0.18434410942925347, 'epoch': 3.0})

In [7]:
# Тестирование
preds = trainer.predict(test_dataset)
print("Test Accuracy:", preds.metrics['test_accuracy'])

D:\dfsd\colab-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test Accuracy: 0.9333333333333333


### Вывод
- Модель DistilBERT успешно классифицировала твиты на два класса.
- Использование токенизатора `distilbert-base-uncased` позволило эффективно обработать тексты.
- Достигнута высокая точность на валидационной и тестовой выборках.
- Hugging Face Transformers значительно упрощают весь ML pipeline для NLP задач.